# Meetup Project


What is in a Meetup API request?

1. https://api.meetup.com: The API host.
2. /find/groups: The method
3. zip=11211&radius=1&category=25: The args. Here is where you may add the API key.
    From what I can tell, the only args you can give are zip, radius, and category. So maybe you need to go through the zips 
4. &order=members:  How to order the results.


-  Private groups I cannot see the members who go, only the number. We will need to see how many groups are public.


# Imports / Pymongo Setup

In [56]:
from pymongo import MongoClient
import requests
import json
key = open('api.txt', 'r').read()

client = MongoClient('localhost', 27017)
db = client.meet_up
collection = db.groups

## Get number of groups in each category globally

In [2]:
# This creates a list of dictionaries, with name, sort_name, id, and shortname. 
dict_categories = json.loads(requests.get('https://api.meetup.com/2/categories?key={}'.format(key)).text)['results']

#make list of ids 
ids = [item['id'] for item in dict_categories]
ids.sort()

In [3]:
def get_count_of_category(number):
    req_count = requests.get('https://api.meetup.com/find/groups?key={}&radius=global&category={}'.format(key, number))\
                        .headers['X-Total-Count']
    cat_name = next(item['name'] for item in dict_categories if item["id"] == number)
    return [number, int(req_count), cat_name]
    
groups_per_category = [get_count_of_category(category) for category in ids]

In [9]:
sum([val[1] for val in groups_per_category])

344808

I find 344,607 groups globally. Wow!

However, I can't seem to get the same number through the json loader. 

Trying with a low group category, Sci-Fi and Fantasy. 

I need to use the offset and page parameter.
Goal for the day: for category 24, get all of its data into mongo. And make experimental design. 

In [58]:
# req = requests.get('https://api.meetup.com/find/groups?key={}&category=24&radius=global&page=403'.format(key))
req = requests.get('https://api.meetup.com/find/groups?page=5&offset=1&key={}'.format(key))

In [59]:
req.headers

{'Content-Type': 'application/json;charset=utf-8', 'Server': 'Apache-Coyote/1.1', 'X-Meetup-server': '4f6999537f81', 'X-Meetup-Request-ID': '36078195-45cf-4709-9d28-22d5bb02079f', 'X-OAuth-Scopes': 'basic', 'X-Accepted-OAuth-Scopes': 'basic', 'X-RateLimit-Limit': '30', 'X-RateLimit-Remaining': '29', 'X-RateLimit-Reset': '10', 'Link': '<https://api.meetup.com/find/groups?page=5&offset=2>; rel="next", <https://api.meetup.com/find/groups?page=5&offset=0>; rel="prev"', 'X-Total-Count': '675', 'ETag': '"e1f511b5f786c5b0b04bac1fd7172779-gzip"', 'Content-Encoding': 'gzip', 'Content-Length': '6694', 'Accept-Ranges': 'bytes', 'Date': 'Mon, 11 Feb 2019 22:04:38 GMT', 'Via': '1.1 varnish', 'Connection': 'keep-alive', 'X-Served-By': 'cache-dfw18645-DFW', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'X-Timer': 'S1549922678.400131,VS0,VE115', 'Vary': 'Accept-Encoding,User-Agent,Accept-Language'}

In [52]:
req.text

'[{"score":1.0,"id":13687982,"name":"Austin User Experience Professionals Association (UXPA)","status":"active","link":"https://www.meetup.com/Austin-User-Experience-Professionals-Association/","urlname":"Austin-User-Experience-Professionals-Association","description":"<p>Austin UXPA is a local chapter of the <a href=\\"http://uxpa.org/\\">User Experience Professionals Association&nbsp;(UXPA)</a><span>.&nbsp; We are a non-profit, volunteer-run 503(c) charity with the goal of providing educational and networking events for user experience designers, researchers, information architects and developers.</span></p>\\n<p><span>We generally meet the first Tuesday of the month in Austin, but the location varies.&nbsp;&nbsp;</span> <br></p>\\n<p><span>Our 2018 Board and Advisory members include:</span>&nbsp; &nbsp;Donna Habersaat, Lawrence Najjar, Danielle Mundle, Ellen Lazaretti, Reyda Taylor, Srikant Chari, Kristin Sullivan, Bart Renner, Kate Moon, Lori Lopez, Kailyn Lim, Erika Grijalva, Lari

In [53]:
len(json.loads(req.text))

5

In [12]:
groups_per_category

[[1, '7093', 'Arts & Culture'],
 [2, '51192', 'Career & Business'],
 [3, '2037', 'Cars & Motorcycles'],
 [4, '7435', 'Community & Environment'],
 [5, '4944', 'Dancing'],
 [6, '9414', 'Education & Learning'],
 [8, '1498', 'Fashion & Beauty'],
 [9, '8218', 'Fitness'],
 [10, '10088', 'Food & Drink'],
 [11, '7803', 'Games'],
 [12, '4365', 'LGBT'],
 [13, '5210', 'Movements & Politics'],
 [14, '27448', 'Health & Wellbeing'],
 [15, '4029', 'Hobbies & Crafts'],
 [16, '14144', 'Language & Ethnic Identity'],
 [17, '1482', 'Lifestyle'],
 [18, '4322', 'Book Clubs'],
 [20, '3500', 'Movies & Film'],
 [21, '7111', 'Music'],
 [22, '17362', 'New Age & Spirituality'],
 [23, '14536', 'Outdoors & Adventure'],
 [24, '403', 'Paranormal'],
 [25, '6938', 'Parents & Family'],
 [26, '2548', 'Pets & Animals'],
 [27, '3880', 'Photography'],
 [28, '6290', 'Religion & Beliefs'],
 [29, '1390', 'Sci-Fi & Fantasy'],
 [30, '2392', 'Singles'],
 [31, '23033', 'Socializing'],
 [32, '9908', 'Sports & Recreation'],
 [33, '6